**Load Raw data From Bronze layer to Sliver Layer**

In [ ]:
# Load raw data into Bronze layer
silver_df_master = spark.read.csv("Files/data/master_dataframe.csv", header=True, inferSchema=True)
silver_df_by_ball = spark.read.csv("Files/data/ipl_ball_by_ball_output_raw.csv", header=True, inferSchema=True)
silver_df_summary = spark.read.csv("Files/data/ipl_summary_raw.csv", header=True, inferSchema=True)


**Dispaly Data Frame data**

In [ ]:
display(silver_df_summary)

**Streamline venue names by removing city names from the venue**

In [ ]:
from pyspark.sql.functions import col, regexp_replace

silver_df_summary_cleaned = silver_df_summary.withColumn(
    "info_venue",
    regexp_replace(col("info_venue"), ",.*$", "")
)

# Show the cleaned venue names
silver_df_summary_cleaned.select("info_venue").distinct().show
display(silver_df_summary_cleaned)

**Transformations for City Name, Team Name, and Stadium Name Changes**

In [8]:
from pyspark.sql.functions import col, when

silver_df_summary_cleaned = silver_df_summary_cleaned.withColumn(
    "info_city", when(col("info_city") == "Bangalore", "Bengaluru")
               .otherwise(col("info_city"))
).withColumn(
    "info_teams_1", when(col("info_teams_1") == "Delhi Daredevils", "Delhi Capitals")
              .when(col("info_teams_1") == "Kings XI Punjab", "Punjab Kings")
              .otherwise(col("info_teams_1"))
).withColumn(
    "info_teams_2", when(col("info_teams_2") == "Delhi Daredevils", "Delhi Capitals")
              .when(col("info_teams_2") == "Kings XI Punjab", "Punjab Kings")
              .otherwise(col("info_teams_2"))
).withColumn(
    "info_toss_winner", when(col("info_toss_winner") == "Delhi Daredevils", "Delhi Capitals")
              .when(col("info_toss_winner") == "Kings XI Punjab", "Punjab Kings")
              .otherwise(col("info_toss_winner"))      
).withColumn(
    "info_outcome_winner", when(col("info_outcome_winner") == "Delhi Daredevils", "Delhi Capitals")
              .when(col("info_outcome_winner") == "Kings XI Punjab", "Punjab Kings")
              .otherwise(col("info_outcome_winner"))            
).withColumn(
    "info_venue", when(col("info_venue") == "M.Chinnaswamy Stadium", "M Chinnaswamy Stadium")
             .otherwise(col("info_venue"))
).na.fill("Unknown", subset=["info_venue", "info_city", "info_teams_1", "info_teams_2", "info_toss_winner","info_outcome_winner"])

silver_df_summary_cleaned.select("info_venue").distinct().show()
#display(silver_df_summary_cleaned)

StatementMeta(, 0595229a-5ef2-441c-8f8b-2fc19bdf8a61, 10, Finished, Available, Finished)

+--------------------+
|          info_venue|
+--------------------+
|Dubai Internation...|
|Himachal Pradesh ...|
|Punjab Cricket As...|
|Vidarbha Cricket ...|
|    Barabati Stadium|
|       Nehru Stadium|
|Maharashtra Crick...|
|        Eden Gardens|
|     OUTsurance Oval|
|M Chinnaswamy Sta...|
|    Feroz Shah Kotla|
|   Brabourne Stadium|
|Barsapara Cricket...|
|          Green Park|
|Holkar Cricket St...|
|Bharat Ratna Shri...|
|Shaheed Veer Nara...|
|Sheikh Zayed Stadium|
|Sharjah Cricket S...|
|    St George's Park|
+--------------------+
only showing top 20 rows



In [ ]:
from pyspark.sql.functions import regexp_replace, trim

# Clean venue names by removing city or extra descriptions after commas
# We want to keep only the main venue name, even if there are commas or additional details
silver_df_summary_cleaned = silver_df_summary_cleaned.withColumn(
    "info_venue",
    trim(
        regexp_replace(col("info_venue"), r",.*$", "")
    )
)

# Manually handle cases where venues might still need extra corrections
# For example, handle specific cases like "MA Chidambaram Stadium" without the city part
venue_cleaning_rules = [
    (r"Rajiv Gandhi International Stadium.*", "Rajiv Gandhi International Stadium"),
    (r"Punjab Cricket Association IS Bindra Stadium.*", "Punjab Cricket Association IS Bindra Stadium"),
    (r"MA Chidambaram Stadium.*", "MA Chidambaram Stadium"),
    # Add any other complex venues here
]

# Apply the specific cleaning rules
for pattern, replacement in venue_cleaning_rules:
    silver_df_summary_cleaned = silver_df_summary_cleaned.withColumn(
        "info_venue",
        regexp_replace(col("info_venue"), pattern, replacement)
    )

# Show the cleaned venue names to verify
silver_df_summary_cleaned.select("info_venue").distinct().show()
display(silver_df_summary_cleaned)

In [11]:
# Create empty table in silver Layer
from delta.tables import DeltaTable

# Create or update Silver table if not exists
silver_table = DeltaTable.createIfNotExists(spark).location("Tables/silver_summary_table").execute()

StatementMeta(, 0595229a-5ef2-441c-8f8b-2fc19bdf8a61, 13, Finished, Available, Finished)

In [12]:
# Write the transformed / Cleaned data into the Silver layer
silver_df_summary_cleaned.write.option("overwriteSchema", "true").format("delta").mode("overwrite").save("Tables/silver_summary_table")  

StatementMeta(, 0595229a-5ef2-441c-8f8b-2fc19bdf8a61, 14, Finished, Available, Finished)